# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np   #C++ module, often used in Finance, used in a faster programming language
import pandas as pd # easier to work with tabular data, rows & columns, dataframe holds tabular data
import requests  # for making HTTP requests, send to an API to get back some data, IEX cloud API
import xlsxwriter # saving excel documents
import math # math functions for operations in a Python script

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head(10)

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
#print(api_url)
data = requests.get(api_url).json()
#print(data['symbol'])
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'EGET ASSSOLKLLEAM DGACAN R/N(T)BQ', 'calculationPrice': 'close', 'open': 138.36, 'openTime': 1657799208863, 'openSource': 'lifaiofc', 'close': 142.61, 'closeTime': 1637569906103, 'closeSource': 'oliifcfa', 'high': 140.97, 'highTime': 1671621230506, 'highSource': 'taee leyueddpi1c ni mr5', 'low': 141.23, 'lowTime': 1649178589431, 'lowSource': 'eepiendyi  dtec1ur5l ma', 'latestPrice': 139.37, 'latestSource': 'Close', 'latestTime': 'February 5, 2021', 'latestUpdate': 1660814380591, 'latestVolume': 76921411, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 138.95, 'delayedPriceTime': 1660685764927, 'oddLotDelayedPrice': 143.54, 'oddLotDelayedPriceTime': 1658464162411, 'extendedPrice': 139.27, 'extendedChange': -0.1, 'extendedChangePercent': -0.00076, 'extendedPriceTime': 1686290281254, 'previousClose': 142.928, 'previousVolume': 86446872, 'change': -0.432, 'changePercent': -0.0032, 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap/1000000000,
        'N/A'
    ],
        index = my_columns),
    
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,139.37,2336.604809,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
#print(api_url)
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index = my_columns),
    ignore_index = True)

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,139.06,2378245468204,N/A
1,AAL,137.47,2323273310638,N/A
2,AAP,139.59,2296478752133,N/A
3,AAPL,139.55,2374687320071,N/A
4,ABBV,139.01,2409104533362,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
# we need to find a way to split our lists of tickers into sublists of list 100
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
# use the chunks function to create a list of lists where every list is no longer than 100

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
 5       ABC
 6      ABMD
 7       ABT
 8       ACN
 9      ADBE
 10      ADI
 11      ADM
 12      ADP
 13     ADSK
 14      AEE
 15      AEP
 16      AES
 17      AFL
 18      AIG
 19      AIV
 20      AIZ
 21      AJG
 22     AKAM
 23      ALB
 24     ALGN
 25      ALK
 26      ALL
 27     ALLE
 28     ALXN
 29     AMAT
       ...  
 70       BR
 71    BRK.B
 72      BSX
 73      BWA
 74      BXP
 75        C
 76      CAG
 77      CAH
 78     CARR
 79      CAT
 80       CB
 81     CBOE
 82     CBRE
 83      CCI
 84      CCL
 85     CDNS
 86      CDW
 87       CE
 88     CERN
 89       CF
 90      CFG
 91      CHD
 92     CHRW
 93     CHTR
 94       CI
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object, 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
 105     CNP
 106     COF
 107     COG
 108     COO
 109     COP
 110    COST
 111    COTY
 112     

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []  # need to transform all stocks that are into the list into a string, string will be passed in to the URL of the HTTP request
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
   # print(symbol_string)
    batch_api_call_url =f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    #print(data.status_code)
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
            
        ],
            index = my_columns),
            ignore_index = True
    )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.12,39565052856,N/A
1,AAL,17.73,10879795015,N/A
2,AAP,164.94,11028562094,N/A
3,AAPL,137.56,2385125062123,N/A
4,ABBV,109.32,196394110119,N/A
5,ABC,110.11,22472132310,N/A
6,ABMD,334.64,15074729303,N/A
7,ABT,124.43,225522341558,N/A
8,ACN,257.40,173914790432,N/A
9,ADBE,508.88,237749095146,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [18]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)
    


Enter the value of your portfolio:10000000


In [19]:
position_size = val/len(final_dataframe.index)
num_apple_shares = position_size/500
print(math.floor(num_apple_shares))

39


In [22]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe   

# num_apple_shares = position_size/500
# print(math.floor(num_apple_shares))

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.12,39565052856,154
1,AAL,17.73,10879795015,1116
2,AAP,164.94,11028562094,120
3,AAPL,137.56,2385125062123,143
4,ABBV,109.32,196394110119,181
5,ABC,110.11,22472132310,179
6,ABMD,334.64,15074729303,59
7,ABT,124.43,225522341558,159
8,ACN,257.40,173914790432,76
9,ADBE,508.88,237749095146,38


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [48]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [49]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
}
)
dollar_format = writer.book.add_format(
    {
    'num_format':'$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    }
)

integer_format = writer.book.add_format(
{
    'num_format': '$0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
}
)



### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [50]:
# writer.sheets['Recommended Trades'].set_columns('A':'A', 18, string_format)
# writer.sheets['Recommended Trades'].set_columns('B':'B', 18, string_format)
# writer.sheets['Recommended Trades'].set_columns('C':'C', 18, string_format)
# writer.sheets['Recommended Trades'].set_columns('D':'D', 18, string_format)

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [53]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0], column_formats[column][1])
                                                   
                                                                   

## Saving Our Excel Output

Saving our Excel file is very easy:

In [54]:
writer.save()